In [1]:
import requests
BASE_URL = 'https://en.wikipedia.org'
resp = requests.get(BASE_URL+'/wiki/List_of_Presidents_of_the_United_States')

In [2]:
from bs4 import BeautifulSoup

In [3]:
soup = BeautifulSoup(resp.text, 'lxml')

In [4]:
tables = soup.select('table')

In [5]:
table_rows = tables[1].select('tr')

## Question 1 - How many Presidents died while in office? Identify them by name, and provide the date of their death.


Strategy -I have picked up the text "(Died in office)" from 2nd column(Startterm and endterm) to identify Presidents who died in office.  


In [6]:
datatotable1 = []

start_row = 2
for row in range(start_row, len(table_rows)):
    data_items = table_rows[row].select('td')
    try:
        Presidentname = data_items[3].find('a').text
        DiedinOffice = data_items[1].text
    except:
        continue
        
    try:
        Died_on = data_items[1].select('span')[1].text
    except:
        try:
            Died_on = data_items[1].text.split('\n')[2]
            # Franklin D. Roosevelt is formatted differently.
        except:
            continue
        

    try:
        if '(Died in office)' in DiedinOffice:
            DiedinOffice = 'Yes' #if 'Died in office' is found in first column then printing 'Yes'
        else:
            DiedinOffice = 'No'
    except:
        continue
    print(Presidentname)
    print(DiedinOffice)
    print(Died_on)
        
    
    datatotable1.append((Presidentname,DiedinOffice,Died_on))

George Washington
No
March 4, 1797
John Adams
No
March 4, 1801
Thomas Jefferson
No
March 4, 1809
James Madison
No
March 4, 1817
James Monroe
No
March 4, 1825
John Quincy Adams
No
March 4, 1829
Andrew Jackson
No
March 4, 1837
Martin Van Buren
No
March 4, 1841
William Henry Harrison
Yes
April 4, 1841
John Tyler
No
March 4, 1845
James K. Polk
No
March 4, 1849
Zachary Taylor
Yes
July 9, 1850
Millard Fillmore
No
March 4, 1853
Franklin Pierce
No
March 4, 1857
James Buchanan
No
March 4, 1861
Abraham Lincoln
Yes
April 15, 1865
Andrew Johnson
No
March 4, 1869
Ulysses S. Grant
No
March 4, 1877
Rutherford B. Hayes
No
March 4, 1881
James A. Garfield
Yes
September 19, 1881
Chester A. Arthur
No
March 4, 1885
Grover Cleveland
No
March 4, 1889
Benjamin Harrison
No
March 4, 1893
Grover Cleveland
No
March 4, 1897
William McKinley
Yes
September 14, 1901
Theodore Roosevelt
No
March 4, 1909
William Howard Taft
No
March 4, 1913
Woodrow Wilson
No
March 4, 1921
Warren G. Harding
Yes
August 2, 1923
Calvin Cool

In [7]:
datatotable1

[('George Washington', 'No', 'March 4, 1797'),
 ('John Adams', 'No', 'March 4, 1801'),
 ('Thomas Jefferson', 'No', 'March 4, 1809'),
 ('James Madison', 'No', 'March 4, 1817'),
 ('James Monroe', 'No', 'March 4, 1825'),
 ('John Quincy Adams', 'No', 'March 4, 1829'),
 ('Andrew Jackson', 'No', 'March 4, 1837'),
 ('Martin Van Buren', 'No', 'March 4, 1841'),
 ('William Henry Harrison', 'Yes', 'April 4, 1841'),
 ('John Tyler', 'No', 'March 4, 1845'),
 ('James K. Polk', 'No', 'March 4, 1849'),
 ('Zachary Taylor', 'Yes', 'July 9, 1850'),
 ('Millard Fillmore', 'No', 'March 4, 1853'),
 ('Franklin Pierce', 'No', 'March 4, 1857'),
 ('James Buchanan', 'No', 'March 4, 1861'),
 ('Abraham Lincoln', 'Yes', 'April 15, 1865'),
 ('Andrew Johnson', 'No', 'March 4, 1869'),
 ('Ulysses S. Grant', 'No', 'March 4, 1877'),
 ('Rutherford B. Hayes', 'No', 'March 4, 1881'),
 ('James A. Garfield', 'Yes', 'September 19, 1881'),
 ('Chester A. Arthur', 'No', 'March 4, 1885'),
 ('Grover Cleveland', 'No', 'March 4, 1889')

In [8]:
from DBcm import UseDatabase

In [9]:
import DBcm

In [10]:
config = { 'host': '127.0.0.1',
           'user': 'kavya',
           'password': 'kavya',
           'database': 'president' }

##### Creating the table for presidents who died in office.

#### Inserting row_items into table 'Question1'

In [282]:
with UseDatabase(config) as cursor:
    SQL = """insert into Question1 
             (Presidentname, DiedinOffice, Died_on)
             values
             (%s, %s, %s)"""
    for row in datatotable1:
        cursor.execute(SQL, row)

#### Query 1) To find out how many died in office.

In [11]:
with UseDatabase(config) as cursor:
    _SQL = "select count(*)  from Question1 where DiedinOffice = 'Yes';"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)
    print(data[0][0],"presidents died in office.")

[(8,)]
8 presidents died in office.


#### Query 2)To find out the President's name and their death date who died in office.

In [283]:
with UseDatabase(config) as cursor:
    _SQL = "select Presidentname,Died_on from Question1 where DiedinOffice = 'Yes';"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)
    

[('William Henry Harrison', 'April 4, 1841'), ('Zachary Taylor', 'July 9, 1850'), ('Abraham Lincoln', 'April 15, 1865'), ('James A. Garfield', 'September 19, 1881'), ('William McKinley', 'September 14, 1901'), ('Warren G. Harding', 'August 2, 1923'), ('Franklin D. Roosevelt', 'April 12, 1945'), ('John F. Kennedy', 'November 22, 1963')]


## Question 2 - How many Presidents resigned the presidency? Identify them by name, and provide the date of their resignation.


Strategy -I have picked up the text "(Resigned from ofice)" from 2nd column(Startterm and endterm) to identify Presidents who Resigned from office. From the rows picked, if "(Resigned from ofice)" is found in 2nd column then 'Yes' is printed. 

In [284]:
datatotable2 = []

start_row = 2
for row in range(start_row, len(table_rows)):
    data_items = table_rows[row].select('td')
    try:
        Presidentname = data_items[3].find('a').text
        Resigned = data_items[1].find('a').text
        Resigned_on = data_items[1].select('span')[1].text
    except:
        continue

    try:
        if 'Resigned from office' in Resigned:
            Resigned = 'Yes' #if 'Resigned' is found in first column then printing 'Yes'
        else:
            Resigned = 'No'
    except:
        continue
    
    datatotable2.append((Presidentname, 
                       Resigned,
                       Resigned_on))   

In [285]:
datatotable2

[('George Washington', 'No', 'March 4, 1797'),
 ('John Tyler', 'No', 'March 4, 1845'),
 ('Millard Fillmore', 'No', 'March 4, 1853'),
 ('Abraham Lincoln', 'No', 'April 15, 1865'),
 ('James A. Garfield', 'No', 'September 19, 1881'),
 ('Chester A. Arthur', 'No', 'March 4, 1885'),
 ('William McKinley', 'No', 'September 14, 1901'),
 ('Calvin Coolidge', 'No', 'March 4, 1929'),
 ('John F. Kennedy', 'No', 'November 22, 1963'),
 ('Richard Nixon', 'Yes', 'August 9, 1974')]

#### Creating table for Presidents who resigned from office.

#### Inserting row items into table 'Question2'

In [286]:
with UseDatabase(config) as cursor:
    SQL = """insert into Question2 
             (Presidentname,Resigned,Resigned_on)
             values
             (%s, %s, %s)"""
    for row in datatotable2:
        cursor.execute(SQL, row)

#### Query 1)How many Presidents Resigned from office.

In [287]:
with UseDatabase(config) as cursor:
    _SQL = "select count(Resigned) from Question2 where Resigned = 'Yes'"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)
    print(data[0][0],"President resigned from office")

[(1,)]
1 President resigned from office


#### Query 2)Name of the President who resigned from office and their resigned date.

In [30]:
with UseDatabase(config) as cursor:
    _SQL = "select * from Question2 where Resigned = 'Yes'"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)
    print(data[0][0],"resigned the office on",data[0][2])

[('Richard Nixon', 'Yes', 'August 9, 1974')]
Richard Nixon resigned the office on August 9, 1974


## Question3 - How many Presidents previously held the office of Vice-President? Identify them by name,and identify the name(s) of the President(s) they were Vice-President to.
                                   

strategy- I have picked up the text "(Vice President)" from 5th column(Prior Office) to identify Presidents who Previously held the office of Vice President. From the rows picked, if "Vice President" is found in 5th column then 'Yes' is printed. 

In [169]:
datatotable3a = []

start_row = 2
for row in range(start_row, len(table_rows)):
    data_items = table_rows[row].select('td')
    try:
        PriorOffice = data_items[4].text
        PrevVPname = data_items[3].find('a').text
    except:
        continue
    try:
        
        if 'Vice President' in PriorOffice:
            PrevVP = 'Yes'
        else:
            PrevVP = 'No'
    except:
        continue
        
    datatotable3a.append((PrevVPname,PrevVP))
                          

In [170]:
datatotable3a

[('George Washington', 'No'),
 ('John Adams', 'Yes'),
 ('Thomas Jefferson', 'Yes'),
 ('James Madison', 'No'),
 ('James Monroe', 'No'),
 ('John Quincy Adams', 'No'),
 ('Andrew Jackson', 'No'),
 ('Martin Van Buren', 'Yes'),
 ('William Henry Harrison', 'No'),
 ('John Tyler', 'Yes'),
 ('James K. Polk', 'No'),
 ('Zachary Taylor', 'No'),
 ('Millard Fillmore', 'Yes'),
 ('Franklin Pierce', 'No'),
 ('James Buchanan', 'No'),
 ('Abraham Lincoln', 'No'),
 ('Andrew Johnson', 'Yes'),
 ('Ulysses S. Grant', 'No'),
 ('Rutherford B. Hayes', 'No'),
 ('James A. Garfield', 'No'),
 ('Chester A. Arthur', 'Yes'),
 ('Grover Cleveland', 'No'),
 ('Benjamin Harrison', 'No'),
 ('Grover Cleveland', 'No'),
 ('William McKinley', 'No'),
 ('Theodore Roosevelt', 'Yes'),
 ('William Howard Taft', 'No'),
 ('Woodrow Wilson', 'No'),
 ('Warren G. Harding', 'No'),
 ('Calvin Coolidge', 'Yes'),
 ('Herbert Hoover', 'No'),
 ('Franklin D. Roosevelt', 'No'),
 ('Harry S. Truman', 'Yes'),
 ('Dwight D. Eisenhower', 'No'),
 ('John F. Ke

#### Creating the table for Presidents who previousy held the office of Vice President.

#### Inserting the rowitems in table 'Question3a'

In [290]:
with UseDatabase(config) as cursor:
    SQL = """insert into Question3a 
             (PrevVPname,PrevVP)
             values
             (%s, %s)"""
    for row in datatotable3a:
        cursor.execute(SQL, row)

#### Query 1)How many Presidents previously held the office of Vice President.

In [971]:
with UseDatabase(config) as cursor:
    _SQL = "select count(PrevVP) from Question3a where PrevVP = 'Yes'"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)
    print(data[0][0],"Presidents previously held the office of Vice President")

[(14,)]
14 Presidents previously held the office of Vice President


#### Query 2)President'S Names who previously held the oddice of Vice President.

In [291]:
with UseDatabase(config) as cursor:
    _SQL = "select PrevVPname from Question3a where PrevVP = 'Yes'"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)

[('John Adams',), ('Thomas Jefferson',), ('Martin Van Buren',), ('John Tyler',), ('Millard Fillmore',), ('Andrew Johnson',), ('Chester A. Arthur',), ('Theodore Roosevelt',), ('Calvin Coolidge',), ('Harry S. Truman',), ('Lyndon B. Johnson',), ('Richard Nixon',), ('Gerald Ford',), ('George H. W. Bush',)]


#### To answer the second half of Question3 ie. to identify the name(s) of the President(s) they were Vice-President to.

In [172]:
def determine_pres_VP(url):
    pres_resp = requests.get(url)
    soup = BeautifulSoup(pres_resp.text, 'lxml')
    pres_table_rows = soup.select('table')
    
    Pname = 'Unknown'
    Pname = soup.findAll('p')
    
     
    if VPname in ['John Tyler','Millard Fillmore','Theodore Roosevelt','Calvin Coolidge','Gerald Ford','Andrew Johnson']:
        Pname = soup.find(text="President").findNext('td').text # finding the 'td' for the header 'President'
    elif VPname in ['Jimmy Carter','Ronald Reagan','George W. Bush','Bill Clinton','Barack Obama','Donald Trump']:
        Pname = 'none' # there are no Presidents for the above VicePresidents so assigning 'none'
    else:
        Pname = soup.find(text="President").findNext('a').text # finding the 'a' for the header 'President'
    
    return(Pname)

In [173]:
datatotable3b = []

start_row = 2
for row in range(start_row, len(table_rows)):
    data_items = table_rows[row].select('td')
    try:
        VPname = data_items[3].find('a').text
    except:
        continue 
    if VPname in ['George Washington', 'Donald Trump', 'John Tyler', 'Millard Fillmore', 'Abraham Lincoln', 'Chester A. Arthur', 'William McKinley', 'Calvin Coolidge', 'John F. Kennedy', 'Richard Nixon']:
        which_anchor = 3
    elif VPname in ['William Henry Harrison', 'Zachary Taylor']:
        which_anchor = 1
    else:
        which_anchor = 2
    more_details = BASE_URL+table_rows[row].select('a')[which_anchor].attrs['href']
    Pname = determine_pres_VP(more_details)
    
    datatotable3b.append((VPname,Pname))

In [174]:
datatotable3b

[('George Washington', 'George Washington'),
 ('John Adams', 'George Washington'),
 ('Thomas Jefferson', 'John Adams'),
 ('James Madison', 'Thomas Jefferson'),
 ('James Monroe', 'James Madison'),
 ('John Quincy Adams', 'James Monroe'),
 ('Andrew Jackson', 'Vice President'),
 ('Martin Van Buren', 'Andrew Jackson'),
 ('William Henry Harrison', 'John Quincy Adams'),
 ('John Tyler', 'William Henry Harrison'),
 ('James K. Polk', 'Andrew Jackson'),
 ('Zachary Taylor', 'Vice President'),
 ('Millard Fillmore', 'Zachary Taylor'),
 ('Franklin Pierce', 'Vice President'),
 ('James Buchanan', 'Joseph Reed Ingersoll'),
 ('Abraham Lincoln', 'Vice President'),
 ('Andrew Johnson', 'Abraham Lincoln'),
 ('Ulysses S. Grant', 'Abraham Lincoln'),
 ('Rutherford B. Hayes', 'Vice President'),
 ('James A. Garfield', 'Vice President'),
 ('Chester A. Arthur', 'James A. Garfield'),
 ('Grover Cleveland', 'Vice President'),
 ('Benjamin Harrison', 'William Henry Harrison'),
 ('Grover Cleveland', 'Vice President'),
 (

#### Creating a table to fetch the name of Presidents of Vice Presidents

#### Inserting the row items in table3b.

In [175]:
with UseDatabase(config) as cursor:
    SQL = """insert into Question3b 
             (VPname,Pname)
             values
             (%s, %s)"""
    for row in datatotable3b:
        cursor.execute(SQL, row)

#### Query 3) To fetch the President name of Vice Presidents.

## Question4 - Calculate the total amount of time (years and months) the office of Vice-President has been vacant since the formation of the republic.

Strategy - Picked up the text 'Office Vacant' from the 9th column then calculating the startterm and endterm of office vacancy for each President after the formation of republican party(Presidents after Abraham Lincoln)

In [158]:
import datetime
from dateutil.relativedelta import relativedelta

datatotable4 = []

start_row = 2
for row in range(start_row, len(table_rows)):
    data_items = table_rows[row].select('td')
    
    try:
        Presi = data_items[3].find('a').text
    except:
        continue
        
    if Presi in ['Chester A. Arthur']:#Vice-Presidents Office was vacant till the time he was president.
        try:
            Vacant_startterm = 'September 19, 1881' #Startterm of presidency = Startterm of office Vacancy
        except:
            continue
        try:
            Vacant_endterm = data_items[1].select('span')[1].text #endterm of presidency = endterm of office Vacancy
        except:
            continue
            
    elif Presi in ['Calvin Coolidge']: #Vice-Presidents Office was vacant till March 4, 1925.
        try:
            Vacant_startterm = data_items[1].select('span')[0].text                                                                        
        except:
            continue
        Vacant_endterm = 'March 4, 1925'
        
    
    elif Presi in ['Andrew Johnson']: #Vice-Presidents Office was vacant till the time he was president.
        try:
            Vacant_startterm = data_items[1].select('span')[0].text #Startterm of presidency = Startterm of office Vacancy
        except:
            continue
        try:
            Vacant_endterm = data_items[1].select('span')[1].text #endterm of presidency = endterm of office Vacancy
        except:
            continue
        
   
    elif Presi in ['Ulysses S. Grant']:#Vice-Presidents Office was vacant from November 22, 1875 till the endterm.
        Vacant_startterm = 'November 22, 1875'
        try:
            Vacant_endterm = data_items[1].select('span')[1].text
        except:
            continue
        
            
    elif Presi in ['Grover Cleveland'] and data_items[1].select('span')[1].text == 'March 4, 1889': #Vice-Presidents Office was vacant from November 25, 1885 till the endterm.
        Vacant_startterm = 'November 25, 1885'
        try:
            Vacant_endterm = data_items[1].select('span')[1].text
        except:
            continue
            
    elif Presi in ['William McKinley']:
        Vacant_startterm = 'November 21, 1899'
        Vacant_endterm = 'March 4,1901'
        
    elif Presi in ['Theodore Roosevelt']: #Vice-Presidents Office was vacant till March 4, 1905.
        try:
            Vacant_startterm = data_items[1].select('span')[0].text
        except:
            continue
        Vacant_endterm = 'March 4, 1905'
        
    elif Presi in ['William Howard Taft']: #Vice-Presidents Office was vacant from October 30, 1912.
        Vacant_startterm = 'October 30, 1912'
        try:
            Vacant_endterm = data_items[1].select('span')[1].text
        except:
            continue
        
    elif Presi in ['Harry S. Truman']: #Vice-Presidents Office was vacant till January 20, 1949.
        try:
            Vacant_startterm = data_items[1].select('span')[0].text
        except:
            continue
        Vacant_endterm = 'January 20, 1949'
        
    elif Presi in ['Lyndon B. Johnson']: #Vice-Presidents Office was vacant till January 20, 1965.
        try:
            Vacant_startterm = data_items[1].select('span')[0].text
        except:
            continue
        Vacant_endterm = 'January 20, 1965'
        
    elif Presi in ['Richard Nixon']: #Vice-Presidents Office was vacant from October 10, 1973 till December 6, 1973.
        Vacant_startterm = 'October 10, 1973'
        Vacant_endterm = 'December 6, 1973'
        
    elif Presi in ['William McKinley']:
        try:
            Vacant_startterm = 'November 21, 1899'
            Vacant_endterm = 'March 4,1901'
        except:
            continue
        
    elif Presi in ['Gerald Ford']:#Vice-Presidents Office was vacant till December 19, 1974.
        try:
            Vacant_startterm = data_items[1].select('span')[0].text
        except:
            continue
        Vacant_endterm = 'December 19, 1974'
        
    else: # Vice-Presidents office wasnt vacant.
        Vacant_startterm = 'Not Vacant'
        Vacant_endterm = 'Not Vacant'
        #pass
        
        
    try: # Relativedelta function to convert years and months to days.
        Start_Term = Vacant_startterm
        End_Term = Vacant_endterm

        def datestr2int(date_str):
            return datetime.datetime.strptime(date_str, '%B %d, %Y')
        

        Start = datestr2int(Start_Term)
        End = datestr2int(End_Term)
        Vacant= End - Start
    except:
        continue
        
   
    datatotable4.append((Vacant.days))

In [159]:
datatotable4

[1419, 468, 1262, 1195, 1267, 125, 1379, 425, 57, 132]

#### To get the total number of days the office was vacant.

In [160]:
TotalVacant = sum(datatotable4)
TotalVacant

7729

In [155]:
Vacant= 7729/365

In [134]:
Vacant 

21.175342465753424

## Question 5 - Identify the youngest serving President. At what age did they become President?

In [13]:
import datetime
from dateutil.relativedelta import relativedelta

datatotable5 = []

start_row = 2
for row in range(start_row, len(table_rows)):
    data_items = table_rows[row].select('td')
    try:
        start_term = data_items[1].select('span')[0].text.split('\n')[0]
    except:
        # Franklin D. Roosevelt is formatted differently.
        try:
            start_term = data_items[1].text.split('\n')[0]
        except:
            continue
    start_term_year = start_term[-4:]
    try:
        year_of_birth = data_items[3].text.split('\n')[1][:4] # birthyear
        if 'Born' in data_items[3].text:    # a special case if the President is still alive
            year_of_birth = data_items[3].text.split('\n')[1][-4:]
    except:
        continue
    
    try:
        PreName = data_items[3].find('a').text
    except:
        continue
        
        
    try: # Relativedelta function to convert string into integer and finding the differnce.
        bday= year_of_birth
        start= start_term_year

        def datestr2int(date_str):
            return datetime.datetime.strptime(date_str, '%Y')

        bdayint = datestr2int(bday)
        startint = datestr2int(start)

        Pyears=relativedelta(startint,bdayint)
    except:
        continue
    
        
    datatotable5.append((PreName,Pyears.years))

In [14]:

datatotable5

[('George Washington', 57),
 ('John Adams', 62),
 ('Thomas Jefferson', 58),
 ('James Madison', 58),
 ('James Monroe', 59),
 ('John Quincy Adams', 58),
 ('Andrew Jackson', 62),
 ('Martin Van Buren', 55),
 ('William Henry Harrison', 68),
 ('John Tyler', 51),
 ('James K. Polk', 50),
 ('Zachary Taylor', 65),
 ('Millard Fillmore', 50),
 ('Franklin Pierce', 49),
 ('James Buchanan', 66),
 ('Abraham Lincoln', 52),
 ('Andrew Johnson', 57),
 ('Ulysses S. Grant', 47),
 ('Rutherford B. Hayes', 55),
 ('James A. Garfield', 50),
 ('Chester A. Arthur', 52),
 ('Grover Cleveland', 48),
 ('Benjamin Harrison', 56),
 ('Grover Cleveland', 56),
 ('William McKinley', 54),
 ('Theodore Roosevelt', 43),
 ('William Howard Taft', 52),
 ('Woodrow Wilson', 57),
 ('Warren G. Harding', 56),
 ('Calvin Coolidge', 51),
 ('Herbert Hoover', 55),
 ('Franklin D. Roosevelt', 51),
 ('Harry S. Truman', 61),
 ('Dwight D. Eisenhower', 63),
 ('John F. Kennedy', 44),
 ('Lyndon B. Johnson', 55),
 ('Richard Nixon', 56),
 ('Gerald For

#### Creating the table for to find the age of all Presidents

Create table Question5 (PreName varchar(80),Pyears varchar(10));

#### Inserting the row items into table Question5

In [15]:
with UseDatabase(config) as cursor:
    SQL = """insert into Question5 
             (PreName,Pyears)
             values
             (%s, %s)"""
    for row in datatotable5:
        cursor.execute(SQL, row)

#### Query 1)To find the president whose age is least

In [19]:
with UseDatabase(config) as cursor:
    _SQL = "select min(Pyears) from Question5 ;"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)

[('43',)]


#### Query 2) To find the name of the Youngest Serving President

In [17]:
with UseDatabase(config) as cursor:
    _SQL = " select * from Question5 where Pyears = '43';"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)
    print(data[0][0],"is the youngest serving President")

[('Theodore Roosevelt', '43')]
Theodore Roosevelt is the youngest serving President


## Question 6 - Identify the oldest serving President. At what age did they leave office (die/resign/retire)?

#### Oldest serving President

#### Since Donald Trump is still in presidency he cant be the oldest serving president, even though he became president at the age of 71 as shown in below query.

In [168]:
with UseDatabase(config) as cursor:
    _SQL = "select max(Pyears) as YoungestPresident from Question5;"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)

[('71',)]


In [171]:
with UseDatabase(config) as cursor:
    _SQL = " select * from Question5 where Pyears = '71';"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)
    print(data[0][0],"is the oldest serving President")
    

[('Donald Trump', '71')]
Donald Trump is the oldest serving President


#### To find the oldest serving President.

Strategy- Finding the difference between the President's Birth year and President's End-term to find the age at which President leave the Presidency.

In [20]:
import datetime
from dateutil.relativedelta import relativedelta

datatotable6 = []

start_row = 2
for row in range(start_row, len(table_rows)):
    data_items = table_rows[row].select('td')
    
    try:
        year_of_birth = data_items[3].text.split('\n')[1][:4]
        if 'Born' in data_items[3].text:   # Special case for President's who are still alive.
            year_of_birth = data_items[3].text.split('\n')[1][-4:]
    except:
        continue
    
    try:
        end_term = data_items[1].select('span')[1].text
        
    except:
        # Franklin D. Roosevelt's entry is formatted differently.
        try:
            end_term = data_items[1].text.split('\n')[2]
        except:
            continue
                
    end_term_year = end_term[-4:]
    
    
    try:
        bday= end_term_year
        end= year_of_birth

        def datestr2int(date_str):
            return datetime.datetime.strptime(date_str, '%Y')
        

        bdayint = datestr2int(bday)
        endint = datestr2int(end)

        Year=relativedelta(bdayint,endint) #President's birthyear - Presidency endterm
    except:
        continue
        
    try:
        PresName = data_items[3].find('a').text
    except:
        continue
    print(PresName,Year.years)
    
    datatotable6.append((PresName,Year.years))

George Washington 65
John Adams 66
Thomas Jefferson 66
James Madison 66
James Monroe 67
John Quincy Adams 62
Andrew Jackson 70
Martin Van Buren 59
William Henry Harrison 68
John Tyler 55
James K. Polk 54
Zachary Taylor 66
Millard Fillmore 53
Franklin Pierce 53
James Buchanan 70
Abraham Lincoln 56
Andrew Johnson 61
Ulysses S. Grant 55
Rutherford B. Hayes 59
James A. Garfield 50
Chester A. Arthur 56
Grover Cleveland 52
Benjamin Harrison 60
Grover Cleveland 60
William McKinley 58
Theodore Roosevelt 51
William Howard Taft 56
Woodrow Wilson 65
Warren G. Harding 58
Calvin Coolidge 57
Herbert Hoover 59
Franklin D. Roosevelt 63
Harry S. Truman 69
Dwight D. Eisenhower 71
John F. Kennedy 46
Lyndon B. Johnson 61
Richard Nixon 61
Gerald Ford 64
Jimmy Carter 57
Ronald Reagan 78
George H. W. Bush 69
Bill Clinton 55
George W. Bush 63
Barack Obama 56


In [249]:
datatotable6

[('George Washington', 65),
 ('John Adams', 66),
 ('Thomas Jefferson', 66),
 ('James Madison', 66),
 ('James Monroe', 67),
 ('John Quincy Adams', 62),
 ('Andrew Jackson', 70),
 ('Martin Van Buren', 59),
 ('William Henry Harrison', 68),
 ('John Tyler', 55),
 ('James K. Polk', 54),
 ('Zachary Taylor', 66),
 ('Millard Fillmore', 53),
 ('Franklin Pierce', 53),
 ('James Buchanan', 70),
 ('Abraham Lincoln', 56),
 ('Andrew Johnson', 61),
 ('Ulysses S. Grant', 55),
 ('Rutherford B. Hayes', 59),
 ('James A. Garfield', 50),
 ('Chester A. Arthur', 56),
 ('Grover Cleveland', 52),
 ('Benjamin Harrison', 60),
 ('Grover Cleveland', 60),
 ('William McKinley', 58),
 ('Theodore Roosevelt', 51),
 ('William Howard Taft', 56),
 ('Woodrow Wilson', 65),
 ('Warren G. Harding', 58),
 ('Calvin Coolidge', 57),
 ('Herbert Hoover', 59),
 ('Franklin D. Roosevelt', 63),
 ('Harry S. Truman', 69),
 ('Dwight D. Eisenhower', 71),
 ('John F. Kennedy', 46),
 ('Lyndon B. Johnson', 61),
 ('Richard Nixon', 61),
 ('Gerald For

#### Creating table to find the oldest serving President.

Create table Question6 (PresName varchar(80),Year varchar(10));

#### Inserting row items into table Question6

In [21]:
with UseDatabase(config) as cursor:
    SQL = """insert into Question6 
             (PresName,Year)
             values
             (%s, %s)"""
    for row in datatotable6:
        cursor.execute(SQL, row)

#### Query1) Finding the Oldest serving President

In [22]:
with UseDatabase(config) as cursor:
    _SQL = "select max(Year) as OldestServing from Question6;"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)

[('78',)]


#### Query 2) Finding the name of Oldest serving President

In [200]:
with UseDatabase(config) as cursor:
    _SQL = "select * from Question6 where Year = '78';"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)
    print(data[0][0],"is the oldest serving President.")

[('Ronald Reagan', '78')]
Ronald Reagan is the oldest serving President.


## Question 7 - Identify the president who served the shortest term in office. How long was that term?


In [96]:
import datetime
from dateutil.relativedelta import relativedelta

datatotable7 = []

start_row = 2
for row in range(start_row, len(table_rows)):
    data_items = table_rows[row].select('td')
    
    
    try:
        start_term = data_items[1].select('span')[0].text
    except:
        # Franklin D. Roosevelt's entry is formatted differently.
        try:
            start_term = data_items[1].text.split('\n')[0]
        except:
            continue
    try:
        end_term = data_items[1].select('span')[1].text
    except:
        # Franklin D. Roosevelt's entry is formatted differently.
        try:
            end_term = data_items[1].text.split('\n')[2]
        except:
            continue
    
    try:
        Start_Term = start_term
        End_Term = end_term

        def datestr2int(date_str):
            return datetime.datetime.strptime(date_str, '%B %d, %Y')
        

        Start = datestr2int(Start_Term)
        End = datestr2int(End_Term)

        short=relativedelta(End,Start)
        Short= End - Start    # Startterm - Endterm
    except:
        continue
        
    try:
        Presi = data_items[3].find('a').text
    except:
        continue
        
    datatotable7.append((Presi,Short.days))

In [97]:
datatotable7

[('George Washington', 2865),
 ('John Adams', 1460),
 ('Thomas Jefferson', 2922),
 ('James Madison', 2922),
 ('James Monroe', 2922),
 ('John Quincy Adams', 1461),
 ('Andrew Jackson', 2922),
 ('Martin Van Buren', 1461),
 ('William Henry Harrison', 31),
 ('James K. Polk', 1461),
 ('Zachary Taylor', 492),
 ('Franklin Pierce', 1461),
 ('James Buchanan', 1461),
 ('Abraham Lincoln', 1503),
 ('Andrew Johnson', 1419),
 ('Ulysses S. Grant', 2922),
 ('Rutherford B. Hayes', 1461),
 ('James A. Garfield', 199),
 ('Grover Cleveland', 1461),
 ('Benjamin Harrison', 1461),
 ('Grover Cleveland', 1461),
 ('William McKinley', 1654),
 ('Theodore Roosevelt', 2728),
 ('William Howard Taft', 1461),
 ('Woodrow Wilson', 2922),
 ('Warren G. Harding', 881),
 ('Herbert Hoover', 1461),
 ('Franklin D. Roosevelt', 4422),
 ('Harry S. Truman', 2840),
 ('Dwight D. Eisenhower', 2922),
 ('John F. Kennedy', 1036),
 ('Lyndon B. Johnson', 1886),
 ('Richard Nixon', 2027),
 ('Gerald Ford', 895),
 ('Jimmy Carter', 1461),
 ('Ron

#### Creating the table to find the President who served the shortest term in office.

#### Inserting row data in the table Question7

In [99]:
with UseDatabase(config) as cursor:
    SQL = """insert into Question7 
             (Presi,Short)
             values
             (%s, %s)"""
    for row in datatotable7:
        cursor.execute(SQL, row)

#### Query 1) To find the least number of days

In [103]:
with UseDatabase(config) as cursor:
    _SQL = "select min(Short) from Question7;"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)

[(31,)]


#### Query 2) To find out the name of the President who served the shotest term

In [104]:
with UseDatabase(config) as cursor:
    _SQL = "select * from Question7 where Short = 31;"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)
    print(data[0][0],"served the shortest term of",data[0][1], "days as President.")

[('William Henry Harrison', 31)]
William Henry Harrison served the shortest term of 31 days as President.


## Question8 - Express the number of Presidents for each party-type as a percentage of the total.

In [ ]:
datatotable8 = []

start_row = 2
for row in range(start_row, len(table_rows)):
    data_items = table_rows[row].select('td')
    
        
    try:
        Party = data_items[6].text.split('\n')[0] # to find the Party of each President
    except:
        continue
    
    try:
        Presi = data_items[3].find('a').text # to fetch the President name
    except:
        continue
    
    
    print(Presi,Party)
    datatotable8.append((Presi,Party))

In [246]:
datatotable8

[('George Washington', 'Unaffiliated'),
 ('John Adams', 'Federalist'),
 ('Thomas Jefferson', 'Democratic-'),
 ('James Madison', 'Democratic-'),
 ('James Monroe', 'Democratic-'),
 ('John Quincy Adams', 'Democratic-'),
 ('Andrew Jackson', 'Democratic'),
 ('Martin Van Buren', 'Democratic'),
 ('William Henry Harrison', 'Whig'),
 ('John Tyler', 'Whig'),
 ('James K. Polk', 'Democratic'),
 ('Zachary Taylor', 'Whig'),
 ('Millard Fillmore', 'Whig'),
 ('Franklin Pierce', 'Democratic'),
 ('James Buchanan', 'Democratic'),
 ('Abraham Lincoln', 'Republican'),
 ('Andrew Johnson', 'National Union'),
 ('Ulysses S. Grant', 'Republican'),
 ('Rutherford B. Hayes', 'Republican'),
 ('James A. Garfield', 'Republican'),
 ('Chester A. Arthur', 'Republican'),
 ('Grover Cleveland', 'Democratic'),
 ('Benjamin Harrison', 'Republican'),
 ('Grover Cleveland', 'Democratic'),
 ('William McKinley', 'Republican'),
 ('Theodore Roosevelt', 'Republican'),
 ('William Howard Taft', 'Republican'),
 ('Woodrow Wilson', 'Democra

#### Creating table to fetch the number of presidents for each party type

In [ ]:
with UseDatabase(config) as cursor:
    SQL = """insert into Question8 
             (Presi,Party)
             values
             (%s, %s)"""
    for row in datatotable8:
        cursor.execute(SQL, row)

#### Query 1) To find the percentage of Unaffiliated party

In [162]:
with UseDatabase(config) as cursor:
    _SQL = "select count(*) as Unaffiliated from Question8 where Party = 'Unaffiliated' ;"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)
    print(data[0][0],"president is from Unaffiliated Party")
Percentage=(1/44)*100
print("Percentage of Unaffiliated party is ",Percentage)

[(1,)]
1 president is from Unaffiliated Party
Percentage of Unaffiliated party is  2.272727272727273


#### Query 2) To find the percentage of Republican party

In [163]:
with UseDatabase(config) as cursor:
    _SQL = "select count(*) as Republican from Question8 where Party = 'Republican';"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)
    print(data[0][0],"presidents are from Republican Party")
Percentage=(19/44)*100
print("Percentage of Republican party is ",Percentage)

[(19,)]
19 presidents are from Republican Party
Percentage of Republican party is  43.18181818181818


#### Query 3) To find the percentage of Democratic-Republican party

In [164]:
with UseDatabase(config) as cursor:
    _SQL = "select count(*) as DemocraticRepublican from Question8 where Party = 'Democratic-';"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)
    print(data[0][0],"presidents are from DemocraticRepublican Party")
Percentage=(4/44)*100
print("Percentage of Democratic-Republican party is ",Percentage)

[(4,)]
4 presidents are from DemocraticRepublican Party
Percentage of Democratic-Republican party is  9.090909090909092


#### Query 4) To find the percentage of Whig party

In [165]:
with UseDatabase(config) as cursor:
    _SQL = "select count(*) as Whig from Question8 where Party = 'Whig';"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)
    print(data[0][0],"presidents are from Whig Party")
Percentage=(4/44)*100
print("Percentage of Whig party is ",Percentage)

[(4,)]
4 presidents are from Whig Party
Percentage of Whig party is  9.090909090909092


#### Query 5) To find the percentage of Federalist party

In [167]:
with UseDatabase(config) as cursor:
    _SQL = "select count(*) as Federalist from Question8 where Party = 'Federalist';"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)
    print(data[0][0],"presidents are from Federalist Party")
Percentage=(1/44)*100
print("Percentage of Federalist party is ",Percentage)

[(1,)]
1 presidents are from Federalist Party
Percentage of Federalist party is  2.272727272727273


#### Query 6) To find the percentage of National Union party

In [168]:
with UseDatabase(config) as cursor:
    _SQL = "select count(*) as NationalUnion from Question8 where Party = 'National Union';"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)
    print(data[0][0],"presidents are from NationalUnion Party")
Percentage=(1/44)*100
print("Percentage of NationalUnion party is ",Percentage)

[(1,)]
1 presidents are from NationalUnion Party
Percentage of NationalUnion party is  2.272727272727273


## Question 9 - Categorize the Presidents by US state (i.e., place-of-birth).

In [176]:
def determine_birth_place(url):
    pres_resp = requests.get(url)
    soup = BeautifulSoup(pres_resp.text, 'lxml')
    pres_table_rows = soup.select('table')
    pres_spans = pres_table_rows[0].select('tr')
    # Find the birth place.
    birth_place = 'New York' 
    for th in pres_spans:
        
        try:
            if 'Born' in th.text:
                birth_place1 = th.text.split('\n')[4] or th.text.split('\n')[3]
                birth_place = birth_place1.split(',')[1] # seperating the text with ',' character
            if 'Virginia' in birth_place: #some of the presidents birthplace is mentioned as 'Colony of Virginia' 
                birth_place = 'Virginia'  # converting it into 'Virginia' to make it common
            elif 'British America' in birth_place: # converting 'British America' into 'North Carolina'
                birth_place = 'North Carolina'
            
        except:
            continue
            
    return(birth_place)

In [177]:
datatotable9 = []

start_row = 2
for row in range(start_row, len(table_rows)):
    data_items = table_rows[row].select('td')
    try:
        Prname = data_items[3].find('a').text
    except:
        continue 
    if Prname in ['George Washington', 'Donald Trump', 'John Tyler', 'Millard Fillmore', 'Abraham Lincoln', 'Chester A. Arthur', 'William McKinley', 'Calvin Coolidge', 'John F. Kennedy', 'Richard Nixon']:
        which_anchor = 3
    elif Prname in ['William Henry Harrison', 'Zachary Taylor']:
        which_anchor = 1
    else:
        which_anchor = 2
    more_details = BASE_URL+table_rows[row].select('a')[which_anchor].attrs['href']
    Birthplace = determine_birth_place(more_details)
    
    print(Prname, ':', Birthplace)
    
    datatotable9.append((Prname,Birthplace))

George Washington : Virginia
John Adams :  Massachusetts
Thomas Jefferson : Virginia
James Madison : Virginia
James Monroe : Virginia
John Quincy Adams :  Massachusetts
Andrew Jackson : North Carolina
Martin Van Buren :  New York
William Henry Harrison : Virginia
John Tyler : Virginia
James K. Polk :  North Carolina
Zachary Taylor : Virginia
Millard Fillmore :  New York
Franklin Pierce :  New Hampshire
James Buchanan :  Pennsylvania
Abraham Lincoln :  Kentucky
Andrew Johnson :  North Carolina
Ulysses S. Grant :  Ohio
Rutherford B. Hayes :  Ohio
James A. Garfield :  Ohio
Chester A. Arthur :  Vermont
Grover Cleveland :  New Jersey
Benjamin Harrison :  Ohio
Grover Cleveland :  New Jersey
William McKinley :  Ohio
Theodore Roosevelt :  New York
William Howard Taft :  Ohio
Woodrow Wilson : Virginia
Warren G. Harding :  Ohio
Calvin Coolidge :  Vermont
Herbert Hoover :  Iowa
Franklin D. Roosevelt :  New York
Harry S. Truman :  Missouri
Dwight D. Eisenhower :  Texas
John F. Kennedy :  Massachus

In [178]:
datatotable9

[('George Washington', 'Virginia'),
 ('John Adams', ' Massachusetts'),
 ('Thomas Jefferson', 'Virginia'),
 ('James Madison', 'Virginia'),
 ('James Monroe', 'Virginia'),
 ('John Quincy Adams', ' Massachusetts'),
 ('Andrew Jackson', 'North Carolina'),
 ('Martin Van Buren', ' New York'),
 ('William Henry Harrison', 'Virginia'),
 ('John Tyler', 'Virginia'),
 ('James K. Polk', ' North Carolina'),
 ('Zachary Taylor', 'Virginia'),
 ('Millard Fillmore', ' New York'),
 ('Franklin Pierce', ' New Hampshire'),
 ('James Buchanan', ' Pennsylvania'),
 ('Abraham Lincoln', ' Kentucky'),
 ('Andrew Johnson', ' North Carolina'),
 ('Ulysses S. Grant', ' Ohio'),
 ('Rutherford B. Hayes', ' Ohio'),
 ('James A. Garfield', ' Ohio'),
 ('Chester A. Arthur', ' Vermont'),
 ('Grover Cleveland', ' New Jersey'),
 ('Benjamin Harrison', ' Ohio'),
 ('Grover Cleveland', ' New Jersey'),
 ('William McKinley', ' Ohio'),
 ('Theodore Roosevelt', ' New York'),
 ('William Howard Taft', ' Ohio'),
 ('Woodrow Wilson', 'Virginia'),


#### Creating the table to Categorize the Presidents by their Birthplace.

In [ ]:
Create table Question9 (Prname varchar(80),Birthplace varchar (80));

#### Inserting the row values into table9

In [179]:
with UseDatabase(config) as cursor:
    SQL = """insert into Question9 
             (Prname, Birthplace)
             values
             (%s, %s)"""
    for row in datatotable9:
        cursor.execute(SQL, row)

#### Query1) To find which President belong to which state.

In [180]:
with UseDatabase(config) as cursor:
    _SQL = "select * from Question9 ;"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    for d in data:
        print(d)
    

('George Washington', 'Virginia')
('John Adams', ' Massachusetts')
('Thomas Jefferson', 'Virginia')
('James Madison', 'Virginia')
('James Monroe', 'Virginia')
('John Quincy Adams', ' Massachusetts')
('Andrew Jackson', 'North Carolina')
('Martin Van Buren', ' New York')
('William Henry Harrison', 'Virginia')
('John Tyler', 'Virginia')
('James K. Polk', ' North Carolina')
('Zachary Taylor', 'Virginia')
('Millard Fillmore', ' New York')
('Franklin Pierce', ' New Hampshire')
('James Buchanan', ' Pennsylvania')
('Abraham Lincoln', ' Kentucky')
('Andrew Johnson', ' North Carolina')
('Ulysses S. Grant', ' Ohio')
('Rutherford B. Hayes', ' Ohio')
('James A. Garfield', ' Ohio')
('Chester A. Arthur', ' Vermont')
('Grover Cleveland', ' New Jersey')
('Benjamin Harrison', ' Ohio')
('Grover Cleveland', ' New Jersey')
('William McKinley', ' Ohio')
('Theodore Roosevelt', ' New York')
('William Howard Taft', ' Ohio')
('Woodrow Wilson', 'Virginia')
('Warren G. Harding', ' Ohio')
('Calvin Coolidge', ' Ver

## Question 10 - Identify which state has produced the most Presidents.


#### Query 1) To find which state produced maximum number of Presidents in decreasing order

In [260]:
with UseDatabase(config) as cursor:
    _SQL = "select count(Birthplace),Birthplace from Question9 group by Birthplace order by count(Birthplace) DESC ;"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    for d in data:
        print(d)

(8, 'Virginia')
(7, 'Ohio')
(5, 'New York')
(4, 'Massachusetts')
(3, 'North Carolina')
(2, 'Vermont')
(2, 'New Jersey')
(2, 'Texas')
(1, 'New Hampshire')
(1, 'Pennsylvania')
(1, 'Kentucky')
(1, 'Iowa')
(1, 'Missouri')
(1, 'California')
(1, 'Nebraska')
(1, 'Georgia')
(1, 'Illinois')
(1, 'Arkansas')
(1, 'Connecticut')
(1, 'Hawaii')


#### Query 2)To find which state has produced maximum number of Presidents

In [261]:
with UseDatabase(config) as cursor:
    _SQL = "select count(Birthplace),Birthplace from Question9 group by Birthplace order by count(Birthplace) DESC LIMIT 1;"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    print(data)
    print(data[0][1],"has produced",data[0][1],"Presidents.")

[(8, 'Virginia')]
Virginia has produced Virginia Presidents.


## Question11 - Identify those state(s) which have produced only one President.

#### Query 1) To find out which states have produced one only President

In [119]:
with UseDatabase(config) as cursor:
    _SQL = "select count(Birthplace),Birthplace from Question9 group by Birthplace order by count(Birthplace) ASC LIMIT 12;"
    cursor.execute(_SQL)
    data = cursor.fetchall()
    for d in data:
        print(d)

(1, 'New Hampshire')
(1, 'Pennsylvania')
(1, 'Kentucky')
(1, 'Iowa')
(1, 'Missouri')
(1, 'California')
(1, 'Nebraska')
(1, 'Georgia')
(1, 'Illinois')
(1, 'Arkansas')
(1, 'Connecticut')
(1, 'Hawaii')


## Question 12 - Provide a list of States which have never produced a President.


In [ ]:
INSERT INTO Question12 VALUES ("Alabama" );
                       
INSERT INTO Question12 VALUES ("Alaska"); 

INSERT INTO Question12 VALUES ("Arizona");

INSERT INTO Question12 VALUES ("Arkansas");

INSERT INTO Question12 VALUES ("California");

INSERT INTO Question12 VALUES ("Colorado");

INSERT INTO Question12 VALUES ("Connecticut");

INSERT INTO Question12 VALUES ("Delaware");

INSERT INTO Question12 VALUES ("Florida");

INSERT INTO Question12 VALUES ("Georgia");

INSERT INTO Question12 VALUES ("Hawaii");

INSERT INTO Question12 VALUES ("Idaho");

INSERT INTO Question12 VALUES ("Illinois");

INSERT INTO Question12 VALUES ("Indiana");

INSERT INTO Question12 VALUES ("Iowa");

INSERT INTO Question12 VALUES ("Kansas");

INSERT INTO Question12 VALUES ("Kentucky");

INSERT INTO Question12 VALUES ("Louisiana");

INSERT INTO Question12 VALUES ("Maine");

INSERT INTO Question12 VALUES ("Maryland");

INSERT INTO Question12 VALUES ("Massachusetts");

INSERT INTO Question12 VALUES ("Michigan");

INSERT INTO Question12 VALUES ("Minnesota");

INSERT INTO Question12 VALUES ("Mississippi");

INSERT INTO Question12 VALUES ("Missouri");

INSERT INTO Question12 VALUES ("Montana");

INSERT INTO Question12 VALUES ("Nebraska");

INSERT INTO Question12 VALUES ("Nevada");

INSERT INTO Question12 VALUES ("New Hampshire");

INSERT INTO Question12 VALUES ("New Jersey");

INSERT INTO Question12 VALUES ("New Mexico");

INSERT INTO Question12 VALUES ("New York");

INSERT INTO Question12 VALUES ("North Carolina");

INSERT INTO Question12 VALUES ("North Dakota");

INSERT INTO Question12 VALUES ("Ohio");

INSERT INTO Question12 VALUES ("Oklahoma");

INSERT INTO Question12 VALUES ("Oregon");

INSERT INTO Question12 VALUES ("Pennsylvania");

INSERT INTO Question12 VALUES ("Rhode Island");

INSERT INTO Question12 VALUES ("South Carolina");

INSERT INTO Question12 VALUES ("South Dakota");

INSERT INTO Question12 VALUES ("Tennessee");

INSERT INTO Question12 VALUES ("Texas");

INSERT INTO Question12 VALUES ("Utah");

INSERT INTO Question12 VALUES ("Vermont");

INSERT INTO Question12 VALUES ("Virginia"); 

INSERT INTO Question12 VALUES ("Washington");

INSERT INTO Question12 VALUES ("West Virginia");

INSERT INTO Question12 VALUES ("Wisconsin");

INSERT INTO Question12 VALUES ("Wyoming");

#### Query 1) To list out all the inserted states

#### Query 2) To list out all the states which hasnt produced a single President.